In [ ]:
!pip install -U langchain

In [ ]:
!pip install langchain-community CSVLoader

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain_google_genai

In [ ]:
import getpass
import os

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass('GEMINI_API_KEY: ')

In [ ]:
from langchain_community.document_loaders import CSVLoader
loader = CSVLoader("/content/new_error_dataset.csv")
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100,chunk_overlap = 20)
splits = text_splitter.split_documents(docs)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# ---- Add your API Key ----
import os
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"

# ---- Gemini Embeddings Model ----
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"   # Available Gemini embedding model
)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipython-input-2694639999.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=splits, # Changed from texts=splits to documents=splits
    embedding=embeddings,
    collection_name="my_collection"
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2
)

In [ ]:
system_prompt = """
You are an AI assistant that answers strictly based on the content retrieved from the PDF document.

RULES:
1. Only answer using the information found in the provided PDF context.
2. Do NOT use your own knowledge or make assumptions.
3. If the answer is not present in the context or is unclear, respond with:
   "No relevant information found in the document."
4. Do NOT hallucinate or fabricate details.
5. Keep answers clear, factual, and directly supported by the retrieved text.
6. Never provide information outside the PDF, even if you know it.
"""

In [ ]:
def rag_answer(query):
    # Step 1: Retrieve relevant chunks
    docs = retriever.invoke(query)

    # Step 2: Build context
    context = "\n\n".join([d.page_content for d in docs])

    # Step 3: Create final user prompt
    final_user_prompt = f"""
You are an assistant that answers strictly based on the provided context.

CONTEXT:
{context}

QUESTION:
{query}

Answer based only on the context. If the answer is not found, say: "Information not available in the provided documents."
"""

    # Create message format (optional style)
    messages = [
        ("system", "Your job is to answer only using the given PDF context."),
        ("human", final_user_prompt)
    ]

    # Convert to plain text for models that don't accept messages format
    formatted_prompt = "\n\n".join(
        f"{role.upper()}:\n{content}" for role, content in messages
    )

    # Step 4: LLM call
    response = llm.invoke(formatted_prompt)

    return response


In [ ]:
rag_answer('what is coolent leak')